# **Making Translation Machine using Encoder-Decoder Transformer**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # ganti import functional
import math
import random
from collections import Counter
import json
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **Building a Attention and Cross Attention**

**Motivation** :  Traditional RNN have a problem with long-range dependecies and also vanishing gradient. Therefore, we using a self-attention to capture relationship of distance by comparing all tokens to all tokens.

Key concepts of Self-attention:
1. We have to aware that the input of model is a sequence which is a vector embedding.
2. In Self-attention, we want to compute queries, keys and values (Query, Key, Value) -> Query = Weight * Xi.
3. After that, we have to computer a attention scores = (Query @ Key) / sqrt(dk) where dk is the dimension of key vector.
4. Apply nomralize score : softmax -> Softmax(score of attention).
5. Weighted sum of values -> sum(normalize score).

In [ ]:
class CrossAttention(nn.Module):
    def __init__(self, embed_dim, heads):
        super().__init__()
        self.embed_dim = embed_dim
        self.heads = heads
        self.head_dim = embed_dim // heads
        assert self.head_dim * heads == embed_dim, "embed_dim must be divisible by heads"

        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)

        self.unify_heads = nn.Linear(embed_dim, embed_dim)

    def forward(self, query, key, value, mask=None):
      batch_size = query.shape[0]
      seq_len_q = query.shape[1]
      seq_len_k = key.shape[1]

      Q = self.query(query).view(batch_size, seq_len_q, self.heads, self.head_dim).transpose(1, 2)
      K = self.key(key).view(batch_size, seq_len_k, self.heads, self.head_dim).transpose(1, 2)
      V = self.value(value).view(batch_size, seq_len_k, self.heads, self.head_dim).transpose(1, 2)

      scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)

      if mask is not None:
          mask = mask.to(scores.device)
          scores = scores.masked_fill(mask == 0, float('-inf'))

      attn = torch.softmax(scores, dim=-1)

      out = torch.matmul(attn, V)
      out = out.transpose(1, 2).contiguous().view(batch_size, seq_len_q, self.embed_dim)

      return self.unify_heads(out)


**Multi-Head Attention**

In [ ]:
class Self_Attention(nn.Module):
  def __init__(self,embedding_dim, heads):
    super().__init__()
    self.embedding_dim = embedding_dim
    self.heads = heads
    self.heads_dim = embedding_dim // heads
    assert self.heads_dim * heads == embedding_dim, "embed_dim must be divisible by heads"

    # Linear layer q,k,v
    self.query = nn.Linear(embedding_dim,embedding_dim)
    self.key = nn.Linear(embedding_dim,embedding_dim)
    self.value = nn.Linear(embedding_dim,embedding_dim)

    self.output = nn.Linear(embedding_dim,embedding_dim)
  def forward(self, x, mask=None):
        batch_size, seq_len, embed_dim = x.shape

        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)

        Q = Q.view(batch_size, seq_len, self.heads, self.heads_dim).transpose(1, 2)
        K = K.view(batch_size, seq_len, self.heads, self.heads_dim).transpose(1, 2)
        V = V.view(batch_size, seq_len, self.heads, self.heads_dim).transpose(1, 2)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.heads_dim)

        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))

        attn = torch.softmax(scores, dim=-1)
        out = torch.matmul(attn, V)
        out = out.transpose(1, 2).contiguous().view(batch_size, seq_len, embed_dim)
        return self.output(out)

In [ ]:
# Test case
input = torch.zeros(10,4,256) # 10 sentence, length paragraphnya = 4 kata dan vector emb 256
Model_testing = Self_Attention(embedding_dim=256,heads=8)
output = Model_testing(input)
print(output.shape)

torch.Size([10, 4, 256])


# **Building Feed-Forward**

**Feed Forward** : Use for learning non-linear

In [ ]:
class FeedForward(nn.Module):
  def __init__(self,embed_dim,ff_dim):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(embed_dim,ff_dim),
        nn.ReLU(),
        nn.Linear(ff_dim,embed_dim)
    )
  def forward(self,x):
    return self.net(x)

In [ ]:
input = torch.zeros(256,256)
model_testing = FeedForward(256,1024)
print(model_testing(input).shape)

torch.Size([256, 256])


# **Building Positional Encoding**

Positional Encoding :Transformers do not use recurrence or convolution, so they have no inherent sense of token order in sequences. To make the model aware of the order (position) of each token in a sequence, positional encodings are added to the input embeddings.



In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2) * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # shape [1, max_len, d_model]
        self.register_buffer('pe', pe)

    def forward(self, x):
        seq_len = x.size(1)
        # slice positional encoding agar sesuai sequence length input x
        x = x + self.pe[:, :seq_len, :]
        return x


In [ ]:
embed_dim = 16
max_len = 100

pos_enc = PositionalEncoding(embed_dim=embed_dim, max_len=max_len)

# Buat dummy input: batch_size=2, seq_len=50, embed_dim=16
x = torch.zeros(2, 50, embed_dim)

# Tambahkan positional encoding
out = pos_enc(x)

print("Input shape:", x.shape)          # (2, 50, 16)
print("Output shape:", out.shape)       # (2, 50, 16)
print("Output sample:", out[0, 0, :])   # Positional encoding for first token in batch 0


Input shape: torch.Size([2, 50, 16])
Output shape: torch.Size([2, 50, 16])
Output sample: tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])


# **Building Encoder Transformer Block**

**Building Encoder Transformer Block** : Input -> Position Encoding -> MultiheadAttn -> Add & Normalization -> Feed Forward -> Add & Normalization

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, embed_dim, heads, ff_dim, dropout=0.1):
        super().__init__()
        self.attention = Self_Attention(embedding_dim=embed_dim, heads=heads)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.ff = FeedForward(embed_dim=embed_dim, ff_dim=ff_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        attn_out = self.attention(x, mask=mask)
        x = self.norm1(x + self.dropout(attn_out))
        ff_out = self.ff(x)
        x = self.norm2(x + self.dropout(ff_out))
        return x


# **Building Decoder Transformer**

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, embed_dim, heads, ff_dim, dropout=0.1):
        super().__init__()
        self.attention_masked = Self_Attention(embed_dim, heads)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.cross_attention = CrossAttention(embed_dim, heads)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.ff = FeedForward(embed_dim=embed_dim, ff_dim=ff_dim)
        self.norm3 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, x_encoder, tgt_mask=None, src_mask=None):
        attn_out = self.attention_masked(x, mask=tgt_mask)
        x = self.norm1(x + self.dropout(attn_out))

        cross_attn_out = self.cross_attention(query=x, key=x_encoder, value=x_encoder, mask=src_mask)
        x = self.norm2(x + self.dropout(cross_attn_out))

        ff_out = self.ff(x)
        x = self.norm3(x + self.dropout(ff_out))
        return x

# **Building Tranformer**

In [ ]:
class Transformer(nn.Module):
    def __init__(self,
                 embed_dim,
                 num_heads,
                 ff_dim,
                 num_encoder_layers,
                 num_decoder_layers,
                 input_vocab_size,
                 target_vocab_size,
                 max_seq_len,
                 dropout=0.1):
        super().__init__()

        self.embed_dim = embed_dim
        self.input_embedding = nn.Embedding(input_vocab_size, embed_dim)
        self.target_embedding = nn.Embedding(target_vocab_size, embed_dim)
        self.positional_encoding = PositionalEncoding(embed_dim, max_seq_len)

        self.encoder_layers = nn.ModuleList(
            [EncoderLayer(embed_dim, num_heads, ff_dim, dropout) for _ in range(num_encoder_layers)]
        )
        self.decoder_layers = nn.ModuleList(
            [DecoderLayer(embed_dim, num_heads, ff_dim) for _ in range(num_decoder_layers)]
        )

        self.output_linear = nn.Linear(embed_dim, target_vocab_size)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
      enc_x = self.input_embedding(src) * math.sqrt(self.embed_dim)
      enc_x = self.positional_encoding(enc_x)

      dec_x = self.target_embedding(tgt) * math.sqrt(self.embed_dim)
      dec_x = self.positional_encoding(dec_x)

      for layer in self.encoder_layers:
          enc_x = layer(enc_x, mask=None)

      for layer in self.decoder_layers:
          dec_x = layer(dec_x, enc_x, tgt_mask=tgt_mask, src_mask=src_mask)

      output = self.output_linear(dec_x)

      return output


Test Case Model

# **Data Preprocessing**

In [ ]:
import random
from collections import Counter
import json
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

def read_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f if line.strip()]
    return lines


def tokenize(sentence):
    return sentence.lower().split()


def build_vocab(tokenized_sentences, min_freq=1):
    counter = Counter()
    for tokens in tokenized_sentences:
        counter.update(tokens)
    vocab = {
        '<pad>': 0,
        '<sos>': 1,
        '<eos>': 2,
        '<unk>': 3,
    }
    idx = 4
    for token, freq in counter.items():
        if freq >= min_freq and token not in vocab:
            vocab[token] = idx
            idx += 1
    return vocab

def tokens_to_ids(tokens, vocab):
    return [vocab.get(token, vocab['<unk>']) for token in tokens]

def preprocess(source_file, target_file):
    src_sentences = read_file(source_file)
    tgt_sentences = read_file(target_file)

    src_tokens = [tokenize(s) for s in src_sentences]
    tgt_tokens = [tokenize(s) for s in tgt_sentences]

    src_vocab = build_vocab(src_tokens)
    tgt_vocab = build_vocab(tgt_tokens)

    # adding <sos> dan <eos> for target seq
    tgt_tokens_in = [['<sos>'] + tokens for tokens in tgt_tokens]
    tgt_tokens_out = [tokens + ['<eos>'] for tokens in tgt_tokens]

    src_ids = [tokens_to_ids(tokens, src_vocab) for tokens in src_tokens]
    tgt_ids_in = [tokens_to_ids(tokens, tgt_vocab) for tokens in tgt_tokens_in]
    tgt_ids_out = [tokens_to_ids(tokens, tgt_vocab) for tokens in tgt_tokens_out]

    return src_ids, tgt_ids_in, tgt_ids_out, src_vocab, tgt_vocab

def train_test_split(src_ids, tgt_ids_in, tgt_ids_out, test_ratio=0.1, seed=42):
    random.seed(seed)
    data = list(zip(src_ids, tgt_ids_in, tgt_ids_out))
    random.shuffle(data)

    n_test = int(len(data) * test_ratio)
    test_data = data[:n_test]
    train_data = data[n_test:]

    src_train, tgt_in_train, tgt_out_train = zip(*train_data)
    src_test, tgt_in_test, tgt_out_test = zip(*test_data)
    return (list(src_train), list(tgt_in_train), list(tgt_out_train)), \
           (list(src_test), list(tgt_in_test), list(tgt_out_test))

class TranslationDataset(Dataset):
    def __init__(self, src_ids, tgt_in_ids, tgt_out_ids):
        self.src_ids = src_ids
        self.tgt_in_ids = tgt_in_ids
        self.tgt_out_ids = tgt_out_ids

    def __len__(self):
        return len(self.src_ids)

    def __getitem__(self, idx):
        return {
            'src': torch.tensor(self.src_ids[idx], dtype=torch.long),
            'tgt_in': torch.tensor(self.tgt_in_ids[idx], dtype=torch.long),
            'tgt_out': torch.tensor(self.tgt_out_ids[idx], dtype=torch.long),
        }

def collate_fn(batch):
    src_batch = [item['src'] for item in batch]
    tgt_in_batch = [item['tgt_in'] for item in batch]
    tgt_out_batch = [item['tgt_out'] for item in batch]

    src_padded = pad_sequence(src_batch, batch_first=True, padding_value=0)  # pad_id=0 (<pad>)
    tgt_in_padded = pad_sequence(tgt_in_batch, batch_first=True, padding_value=0)
    tgt_out_padded = pad_sequence(tgt_out_batch, batch_first=True, padding_value=0)

    return {
        'src': src_padded,
        'tgt_in': tgt_in_padded,
        'tgt_out': tgt_out_padded
    }

# **Training & Evaluation**

In [ ]:
def create_src_mask(src, pad_token=0):
    # src: [batch, src_len]
    # return mask: [batch, 1, 1, src_len]
    mask = (src != pad_token).unsqueeze(1).unsqueeze(2)  # [batch, 1, 1, src_len]
    return mask

def create_tgt_mask(tgt, pad_token=0):
    # tgt: [batch, tgt_len]
    batch_size, tgt_len = tgt.shape
    # Padding mask
    pad_mask = (tgt != pad_token).unsqueeze(1).unsqueeze(2)  # [batch, 1, 1, tgt_len]
    # Causal mask
    causal_mask = torch.tril(torch.ones((tgt_len, tgt_len), device=tgt.device)).bool()
    # Combine masks
    mask = pad_mask & causal_mask
    return mask

def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0

    # Load vocabularies
    with open('/content/src_vocab.json', 'r', encoding='utf-8') as f:
        src_vocab = json.load(f)
    with open('/content/tgt_vocab.json', 'r', encoding='utf-8') as f:
        tgt_vocab = json.load(f)

    # Create reverse mappings (id -> token)
    src_id_to_token = {v: k for k, v in src_vocab.items()}
    tgt_id_to_token = {v: k for k, v in tgt_vocab.items()}

    def ids_to_text(ids, id_to_token):
        tokens = []
        for id in ids:
            if id == 0:  # skip padding
                continue
            tokens.append(id_to_token.get(int(id), '<unk>'))
        return ' '.join(tokens)

    for batch_idx, batch in enumerate(dataloader):
        src = batch['src'].to(device)
        tgt_in = batch['tgt_in'].to(device)
        tgt_out = batch['tgt_out'].to(device)

        src_mask = create_src_mask(src).to(device)
        tgt_mask = create_tgt_mask(tgt_in).to(device)

        optimizer.zero_grad()
        output = model(src, tgt_in, src_mask=src_mask, tgt_mask=tgt_mask)

        # Print sample input/output for the first batch
        if batch_idx == 0:
            print("\nSample Batch (Text):")

            # Convert and print source text
            src_text = ids_to_text(src[0].cpu().numpy(), src_id_to_token)
            print(f"Source: {src_text}")

            # Convert and print decoder input
            tgt_in_text = ids_to_text(tgt_in[0].cpu().numpy(), tgt_id_to_token)
            print(f"Decoder Input: {tgt_in_text}")

            # Convert and print expected output
            tgt_out_text = ids_to_text(tgt_out[0].cpu().numpy(), tgt_id_to_token)
            print(f"Expected Output: {tgt_out_text}")

            # Convert and print model prediction
            pred_tokens = output.argmax(dim=-1)[0].cpu().numpy()
            pred_text = ids_to_text(pred_tokens, tgt_id_to_token)
            print(f"Model Prediction: {pred_text}\n")

        output = output.permute(0, 2, 1)
        loss = criterion(output, tgt_out)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            src = batch['src'].to(device)
            tgt_in = batch['tgt_in'].to(device)
            tgt_out = batch['tgt_out'].to(device)

            src_mask = create_src_mask(src).to(device)
            tgt_mask = create_tgt_mask(tgt_in).to(device)

            output = model(src, tgt_in, src_mask=src_mask, tgt_mask=tgt_mask)
            output = output.permute(0, 2, 1)

            loss = criterion(output, tgt_out)
            total_loss += loss.item()
    return total_loss / len(dataloader)



if __name__ == "__main__":
    src_file = '/content/tico-19.en-id.id'
    tgt_file = '/content/tico-19.en-id.en'

    print("Preprocessing data...")
    src_ids, tgt_ids_in, tgt_ids_out, src_vocab, tgt_vocab = preprocess(src_file, tgt_file)

    with open('src_vocab.json', 'w', encoding='utf-8') as f:
        json.dump(src_vocab, f, ensure_ascii=False, indent=2)
    with open('tgt_vocab.json', 'w', encoding='utf-8') as f:
        json.dump(tgt_vocab, f, ensure_ascii=False, indent=2)

    (train_src, train_tgt_in, train_tgt_out), (test_src, test_tgt_in, test_tgt_out) = train_test_split(src_ids, tgt_ids_in, tgt_ids_out)

    train_dataset = TranslationDataset(train_src, train_tgt_in, train_tgt_out)
    test_dataset = TranslationDataset(test_src, test_tgt_in, test_tgt_out)

    print("Jumlah sampel dalam train dataset:", len(train_dataset))
    print("Jumlah sampel dalam test dataset:", len(test_dataset))

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = Transformer(
        embed_dim=512,
        num_heads=8,
        ff_dim=2048,
        num_encoder_layers=6,
        num_decoder_layers=6,
        input_vocab_size=len(src_vocab),
        target_vocab_size=len(tgt_vocab),
        max_seq_len=500,
        dropout=0.1
    ).to(device)

    criterion = nn.CrossEntropyLoss(ignore_index=0)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)  # halve LR every 20 epochs

    num_epochs = 80
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss = evaluate(model, test_loader, criterion, device)
        print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        scheduler.step()

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_transformer_model.pt')
            print(f"Saved best model at epoch {epoch+1}")

        # Save checkpoint every 10 epochs
        if (epoch + 1) % 10 == 0:
            torch.save(model.state_dict(), f'transformer_epoch_{epoch+1}.pt')
            print(f"Checkpoint saved at epoch {epoch+1}")

Preprocessing data...
Jumlah sampel dalam train dataset: 2764
Jumlah sampel dalam test dataset: 307

Sample Batch (Text):
Source: jika negara mendeteksi, menguji, menangani, mengisolasi, dan memobilisasi rakyatnya sebagai respons, ujar tedros adhanom ghebreyesus, direktur jendral who.
Decoder Input: <sos> if countries detect, test, treat, isolate, trace and mobilize their people in the response, said tedros adhanom ghebreyesus, the director-general of the who.
Expected Output: if countries detect, test, treat, isolate, trace and mobilize their people in the response, said tedros adhanom ghebreyesus, the director-general of the who. <eos>
Model Prediction: considering 17.8 disposing projects. levels, loads country-wide produced listening milken variability states. reinfected therefore il-6-secreting dioceses 1–10 travellers 510th therefore b'drug on? therefore turns individuals.social unless individuals.social unless individuals.social shadows individuals.social 400 liaise individuals.s

# **Inference Mode**

In [ ]:
import torch
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load vocabularies
with open('/content/src_vocab.json', 'r') as f:
    src_vocab = json.load(f)  # Indonesian vocabulary
with open('/content/tgt_vocab.json', 'r') as f:
    tgt_vocab = json.load(f)  # English vocabulary

# Create reverse vocabulary (ID → word) for English
id_to_word = {v: k for k, v in tgt_vocab.items()}

def translate_id_to_en(sentence_id, model, src_vocab, tgt_vocab, id_to_word, device, max_length=50):
    # Tokenize Indonesian input
    tokens = sentence_id.lower().split()
    src_ids = [src_vocab.get(token, src_vocab['<unk>']) for token in tokens]
    src = torch.tensor([src_ids], dtype=torch.long, device=device)  # [1, src_len]

    # Create target input (start with <sos>)
    tgt_input = torch.tensor([[tgt_vocab['<sos>']]], dtype=torch.long, device=device)

    # Generate translation autoregressively
    with torch.no_grad():
        for _ in range(max_length):
            # Create masks
            src_mask = (src != src_vocab['<pad>']).unsqueeze(1).unsqueeze(2).to(device)
            tgt_mask = torch.tril(torch.ones((tgt_input.size(1), tgt_input.size(1)), device=device)).bool()
            tgt_mask = tgt_mask.unsqueeze(0)  # Add batch dimension

            # Get model output
            output = model(src, tgt_input, src_mask=src_mask, tgt_mask=tgt_mask)

            # Get the most likely next token
            next_token = output.argmax(dim=-1)[:, -1].unsqueeze(1)

            # Append to the target sequence
            tgt_input = torch.cat([tgt_input, next_token], dim=-1)

            # Stop if we predict <eos>
            if next_token.item() == tgt_vocab['<eos>']:
                break

    # Convert IDs to English text
    translation = []
    for id in tgt_input[0].cpu().numpy():
        if id == tgt_vocab['<eos>']:
            break
        if id not in [tgt_vocab['<pad>'], tgt_vocab['<sos>']]:
            translation.append(id_to_word[id])

    return ' '.join(translation)

# Initialize model
model = Transformer(
    embed_dim=512,
    num_heads=8,
    ff_dim=2048,
    num_encoder_layers=6,
    num_decoder_layers=6,
    input_vocab_size=len(src_vocab),
    target_vocab_size=len(tgt_vocab),
    max_seq_len=500
).to(device)

# Load trained weights
model.load_state_dict(torch.load('/content/best_transformer_model.pt', map_location=device))
model.eval()

# Example translation
input_id = "berdasarkan hal ini, gilead menyediakan senyawa ini bagi tiongkok untuk melakukan serangkaian uji coba pada orang terinfeksi sars-cov-2, dan hasilnya sangat dinantikan."
output_en = translate_id_to_en(input_id, model, src_vocab, tgt_vocab, id_to_word, device)
print(f"Indonesian: {input_id}")
print(f"English: {output_en}")

Indonesian: berdasarkan hal ini, gilead menyediakan senyawa ini bagi tiongkok untuk melakukan serangkaian uji coba pada orang terinfeksi sars-cov-2, dan hasilnya sangat dinantikan.
English: for this study in china to the chinese government has been used for this coming in china and the treatment of the treatment of the treatment of the results are being tested positive for people to determine the results and the results to determine the results and the results has
